# BTAP Data Preprocessing

This notebook preprocess the btap data: 
- Two excel files with building characteristics, one for electricity and the other for gas.
- Two csv files with electricity and gas hourly measurements.
- Buildings have an unique identifier `:datapoint_id`.
- The preprocessing follows btap project process and cleaning steps.

In [ ]:
from datetime import datetime, timedelta

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

## Inputs
- Number of days to generate

In [ ]:
days = timedelta(days=1)

### Read original BTAP data
- Building files

In [ ]:
PATH = "../data/BTAP_Share/"

In [ ]:
path_elec = PATH + "calgary_toronto_elec.xlsx"
path_gas = PATH + "calgary_toronto_gas.xlsx"

In [ ]:
%%time
btap_df = pd.read_excel(path_elec)

In [ ]:
%%time
btap_df = pd.concat([btap_df, pd.read_excel(path_gas)], ignore_index=True)

In [ ]:
btap_df.head()

In [ ]:
btap_df.columns

## Data cleaning

In [ ]:
# Building meters squared
floor_sq = btap_df['bldg_conditioned_floor_area_m_sq'].unique()
#print(btap_df['bldg_conditioned_floor_area_m_sq'].unique())

In [ ]:
# Unique weather keys
epw_keys = btap_df[':epw_file'].unique()
# Dynamic list of columns to remove
output_drop_list = ['Unnamed: 0', ':template']
# List of columns to keep despite being ruled to be removed
output_drop_list_exceptions = ['energy_eui_additional_fuel_gj_per_m_sq',
                               'energy_eui_electricity_gj_per_m_sq',
                               'energy_eui_natural_gas_gj_per_m_sq',
                               'net_site_eui_gj_per_m_sq',
                               ':building_type',
                               ':epw_file',
                               'bldg_conditioned_floor_area_m_sq',
                               ':erv_package'
                              ]

# Since :srr_set contains string and float values, we replace it with
# TODO: Remove when the inputs handle default values
btap_df[':srr_set'] = btap_df['bldg_srr'] / 100

# Remove columns without a ':' and which are not exceptions
for col in btap_df.columns:
    if ((':' not in col) and (col not in output_drop_list_exceptions)):
        output_drop_list.append(col)
btap_df = btap_df.drop(output_drop_list, axis=1)

# Define a Total energy column
if 'net_site_eui_gj_per_m_sq' in btap_df:
    btap_df['Total Energy'] = btap_df[['net_site_eui_gj_per_m_sq']].sum(axis=1)

drop_list = ['energy_eui_additional_fuel_gj_per_m_sq',
             'energy_eui_electricity_gj_per_m_sq',
             'energy_eui_natural_gas_gj_per_m_sq',
             'net_site_eui_gj_per_m_sq',
             ':analysis_id',
             ':analysis_name',
             ':os_standards_branch',
             ':btap_costing_branch']
# Drop any remaining fields which exist, ignoring raised errors
btap_df = btap_df.drop(drop_list, axis=1, errors='ignore')

In [ ]:
btap_df.head()

In [ ]:
def clean_data(df) -> pd.DataFrame:
    """
    Basic cleaning of the data using the following criterion:

    - dropping any column with more than 50% missing values
      The 50% threshold is a way to eliminate columns with too much missing values in the dataset.
      We cant use N/A as it will elimnate the entire row /datapoint_id. Giving the number of features we have to work it its better we eliminate
      columns with features that have too much missing values than to eliminate by rows, which is what N/A will do .
    - dropping columns with 1 unique value
      For columns with  1 unique values are dropped during data cleaning as they have low variance
      and hence have little or no significant contribution to the accuracy of the model.

    Args:
        df: dataset to be cleaned

    Returns:
        df: cleaned dataframe
    """
    # Needed to avoid SettingWithCopyWarning from pandas
    df = df.copy()
    # Drop any column with more than 50% missing values
    half_count = len(df) / 2
    df = df.dropna(thresh=half_count, axis=1)

    # Lists of columns which ignore the one unique value restraint since
    # they may be needed later on
    column_exceptions = ['energy_eui_additional_fuel_gj_per_m_sq',
                         'energy_eui_electricity_gj_per_m_sq',
                         'energy_eui_natural_gas_gj_per_m_sq',
                         ':building_type',
                         ':epw_file',
                         'bldg_conditioned_floor_area_m_sq',
                         'Name']

    # Again, there may be some columns with more than one unique value, but one
    # value that has insignificant frequency in the data set.
    for col in df.columns:
        num = len(df[col].unique())
        # Remove any columns with only one unique value and which are not exceptions
        if ((len(df[col].unique()) == 1) and (col not in column_exceptions)):
            df.drop(col, inplace=True, axis=1)
    return df

## Read original energy data

In [ ]:
path_elec = PATH + "calgary_toronto_elec_energy.csv"
path_gas = PATH + "calgary_toronto_gas_energy.csv"

In [ ]:
%%time
energy_df = pd.read_csv(path_elec)
#if path_gas:
energy_df = pd.concat([energy_df, pd.read_csv(path_gas)], ignore_index=True)
# Adds all except Electricity:Facility
#energy_df.loc[(energy_df['Name'] != "ElectricityNet:Facility") & (energy_df['Name'] != "NaturalGas:Facility"), ['Name']] = "Electricity:Facility"
energy_df = energy_df.loc[(energy_df['Name'] == "ElectricityNet:Facility") | (energy_df['Name'] == "NaturalGas:Facility")]
#print(energy_df[energy_df['Name'] == "ElectricityNet:Facility" or energy_df['Name'] == 'NaturalGas:Facility'])
#energy_df = energy_df[energy_df['Name'] != "Electricity:Facility"].groupby(['datapoint_id']).sum()
# TODO: REMOVE
#energy_df = energy_df.agg(lambda x: x / (floor_sq * 1000000))
energy_df = energy_df.drop(['KeyValue'], axis=1)

## Data cleaning

In [ ]:
%%time
# Clean the energy data
energy_df = clean_data(energy_df)

In [ ]:
energy_df = energy_df.reset_index(drop=True)
# Change the dataframe from having all dates as columns to having
# each current row contain an entry for each date

In [ ]:
energy_df

In [ ]:
len(energy_df)

In [ ]:
#en_copy = energy_df.copy()

In [ ]:
#energy_df = en_copy[:1000]

## Merge building and energy data
- Choose `start_date` and `end_date` for time series
- Check data

In [ ]:
%%time
# Note that this takes a long time to process
energy_df = energy_df.melt(id_vars=['datapoint_id', 'Name'], var_name='Timestamp', value_name='energy')

In [ ]:
energy_df.head(2), energy_df.info()

In [ ]:
%%time
energy_df['Timestamp']= pd.to_datetime(energy_df['Timestamp'])
energy_df.info()

In [ ]:
start_date = energy_df.Timestamp[0]
start_date

In [ ]:
end_date = start_date + days
end_date

In [ ]:
dates = pd.date_range(start=start_date, end=end_date, freq='1h', inclusive='left')
dates

In [ ]:
%%time
energy_in_df = energy_df[energy_df['Timestamp'].isin(dates)]
energy_in_df

In [ ]:
%%time
# Merge gas and electricity rows together
energy_in_df = pd.merge(energy_in_df.loc[(energy_in_df['Name'] == "ElectricityNet:Facility")],
                     energy_in_df.loc[(energy_in_df['Name'] == "NaturalGas:Facility")],
                     on=['datapoint_id', 'Timestamp'],
                     how='outer',
                     suffixes=['_elec', '_gas'])

In [ ]:
%%time
# Remove unused column names
energy_in_df = energy_in_df.drop(["Name_elec", "Name_gas"], axis='columns', errors='ignore')

In [ ]:
%%time    
# Replace blank values with 0 (ex: when there is electricity but no gas)
energy_in_df = energy_in_df.replace(np.nan, 0.0)

In [ ]:
%%time
# Also track the total energy usage
energy_in_df['energy'] = energy_in_df['energy_elec'] + energy_in_df['energy_gas']

In [ ]:
energy_in_df.describe()

In [ ]:
energy_in_df.head(-5)

In [ ]:
nrg_unique_ids = energy_in_df.datapoint_id.unique()
print(len(nrg_unique_ids))

In [ ]:
btap_unique_ids = btap_df[":datapoint_id"].unique()
print(len(btap_unique_ids))

In [ ]:
btap_unique_ids

In [ ]:
btap_in_df = btap_df.loc[btap_df[':datapoint_id'].isin(nrg_unique_ids)]

In [ ]:
btap_in_df = btap_in_df.set_index(':datapoint_id')

In [ ]:
btap_in_df = btap_in_df.reindex(index=nrg_unique_ids)

In [ ]:
btap_in_df = btap_in_df.reset_index()

In [ ]:
btap_in_df[':datapoint_id']

In [ ]:
nrg_unique_ids

In [ ]:
# Sanity check #1
btap_in_df[':datapoint_id'].equals(pd.Series(nrg_unique_ids))

In [ ]:
energy_in_df["datapoint_id"]

In [ ]:
energy_in_df.isna()

In [ ]:
btap_in_df.columns

In [ ]:
ignore_cols = pd.read_csv("costing_columns_to_ignore.txt", header=None)[0].values.tolist()

In [ ]:
if ":datapoint_id" in ignore_cols:
    ignore_cols.remove(":datapoint_id")

In [ ]:
btap_in_df = btap_in_df.drop(columns=ignore_cols, errors='ignore')

In [ ]:
btap_in_df

In [ ]:
btap_in_df.isnull().sum().sum()

In [ ]:
energy_in_sdv_df = energy_in_df.sort_values(by=['datapoint_id', 'Timestamp']).reset_index(drop=True).drop(columns=['energy'])

In [ ]:
energy_in_sdv_df

In [ ]:
btap_in_df = btap_in_df.sort_values(by=[':datapoint_id']).reset_index(drop=True)

In [ ]:
%%time
# time series and static values together
data_in_df = pd.merge(energy_in_sdv_df,
                      btap_in_df,
                      left_on=['datapoint_id'],
                      right_on=[':datapoint_id'],
                      how='left').drop(":datapoint_id", axis=1).reset_index(drop=True)

In [ ]:
data_in_df = data_in_df[data_in_df.columns[:-1]]

In [ ]:
data_in_df

## Save real data

The main output of this notebook. This is the real data we want to synthesize.

In [ ]:
data_in_df.to_csv(f"real_data_sdv_{days.days}_days.csv")

## Done!